In [1]:
# !/python:
#
# imports:
import time
from selenium import webdriver
# requires to install geckodriver in the PATH
# downloaded from https://github.com/…ses
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import cv2


In [2]:

def open_book_url_and_login(url=''):
    """Automatically open the login screen, enter the credentials manually.
    return selenium browser instance
    """
    if url=='':
        url = 'https://archive.org/details/musicoftroubadou0000aubr/mode/1up'


    login = '***'
    pwd = '***'
    height = 2000
    width = 1000

    # initialization: opening the web page
    browser = webdriver.Firefox()

    browser.set_window_size(width,height)
    browser.get(url)

    # login
    btns = browser.find_elements_by_class_name('button')
    btns[0].click() # click the login button
    # you can uncoment the lines for automatic login.
    # please be aware of security risks of writing you login details into the source code
    #uname_box = browser.find_element_by_name('username')
    #uname_box.click()
    #uname_box.send_keys(login)
    #psswd_box = browser.find_element_by_name('password')
    #psswd_box.send_keys(pwd)
    #log_btn = browser.find_element_by_name('submit-to-login')
    #log_btn.click()
    return browser

In [3]:
def get_pageScreenshotCompleted(BRScreen,trial_times=50,sleep_seconds=0.5):
    """Check the variance of the pixels in the last row of the page screenshot.
    If the variance > 0, the page was loaded completely, return, else wait
    trial_times : how many times to try and wait
    sleep_seconds : how many seconds to wait in single cycle before testing variance
    returns the screenshot of the page on PNG format
    """
    #trial_times = 50
    for trial in range(trial_times):
        shot = BRScreen.screenshot_as_png
        img_array = np.frombuffer(shot,dtype=np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        variance_last_row = np.var(np.sum(img[-1,1:-1,:],axis=1))
        print('.',end='' )
        if variance_last_row > 0:
            break
        else:
            time.sleep(sleep_seconds) # additional wait
    return shot

def download_book(browser,height=3000,width=2000):
    """ downloads the book page by page starting from the actual page in the book.  
    Check that it should be zeroth page (scan of the front cover).
    The book should be borrowed already, no automatic borrowing is implemented yet.
    height, width : pixel size of the browser window used for downloading. 
    Sometimes the window accidentally snaps to the screen edges. 
    It should be avoided in the future... 
    return list of the page filenames, as they were downloaded
    """
    # definign the sections of the bookreader:
    #height = 3000
    #width = 2000

    # initialization: opening the web page

    browser.set_window_size(width,height)
    BRcontainer = browser.find_element_by_class_name('BRcontainer')
    BRControls = browser.find_element_by_class_name('BRcontrols')

    BRC_nextPage = BRControls.find_element_by_class_name('BRicon.book_right')


    BRC_nextPage = BRControls.find_element_by_class_name('BRicon.book_right')

    # first page
    pagesFnameLst = []
    i=0
    BRpagecontainerClassName = ("BRmode1up.pagediv%d.protected" % i)
    actual_page = browser.find_element_by_class_name(BRpagecontainerClassName)
    BRScreen = actual_page.find_element_by_class_name('BRscreen')
    shot = get_pageScreenshotCompleted(BRScreen)
    page_fname = ('page%04d.png' % i)
    with open(page_fname,mode='wb') as imgFile:
        imgFile.write(shot)
        imgFile.close()
    pagesFnameLst.append(page_fname)


    while True: 
        BRC_nextPage.click() 
        i+=1
        BRpagecontainerClassName = ("BRmode1up.pagediv%d.protected" % i)
        try:
            #print(WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CLASS_NAME, BRpagecontainerClassName))))
            WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CLASS_NAME, BRpagecontainerClassName)))
        except TimeoutException:
            break
        actual_page = browser.find_element_by_class_name(BRpagecontainerClassName)
        BRScreen = actual_page.find_element_by_class_name('BRscreen')
        # here we need to look at the image, and find out whether it has already loaded
        # if not, i.e. the last row of pixels are uniform - pale yelow, wait another few ms...
        # and take the screenshot again, then it will be foolproof.
        shot = get_pageScreenshotCompleted(BRScreen)
        page_fname = ('page%04d.png' % i)
        print('writing %s' % page_fname)
        with open(page_fname,mode='wb') as imgFile:
            imgFile.write(shot)
            imgFile.close()
        pagesFnameLst.append(page_fname)
    print('finished download, pagecount: %d' % i)
    
    return pagesFnameLst


# execute the functions

In [4]:
cd G:\ML\books_archive_org\scans

G:\ML\books_archive_org\scans


# book https://archive.org/details/nortonscoresanth0000unse/mode/1up

In [ ]:

bookId = 'nortonscoresanth0000unse'
browser = open_book_url_and_login('https://archive.org/details/%s/mode/1up' % bookId)
# we need to borrow the book either before downloading, or 
# do it automatically
time.sleep(60) # wait for some time for user to enter his credentials
pageFnames = download_book (browser)
browser.close()

......writing page0001.png
.writing page0002.png
....writing page0003.png
.writing page0004.png
.......writing page0005.png
.writing page0006.png
........writing page0007.png
.writing page0008.png
.......writing page0009.png
.writing page0010.png
........writing page0011.png
.writing page0012.png
....writing page0013.png
.writing page0014.png
.....writing page0015.png
..writing page0016.png
...writing page0017.png
...writing page0018.png
.writing page0019.png
.......writing page0020.png
.writing page0021.png
......writing page0022.png
.writing page0023.png
.....writing page0024.png
.writing page0025.png
......writing page0026.png
.writing page0027.png
.....writing page0028.png
.writing page0029.png
.....writing page0030.png
.writing page0031.png
.....writing page0032.png
.writing page0033.png
......writing page0034.png
.writing page0035.png
.....writing page0036.png
.writing page0037.png
......writing page0038.png
.writing page0039.png
......writing page0040.png
.writing page0041.png
.

The book should be now dowloaded. 